In [20]:
from __future__ import absolute_import
from __future__ import print_function

import numpy as np
import pandas as pd
import math
import ast

import h5py
import sys
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [21]:

def load_data(path, train=True):
    df = pd.read_csv(path)
    X = df.values.copy()
    if train:
        np.random.shuffle(X)  
        X, labels = X[:, 1:-1].astype(np.float32), X[:, -1]
        return X, labels
    else:
        X, ids = X[:, 1:].astype(np.float32), X[:, 0].astype(str)
        return X, ids


def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler


def preprocess_labels(labels, encoder=None, categorical=True):
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
    y = encoder.transform(labels).astype(np.int32)
    if categorical:
        y = np_utils.to_categorical(y)
    return y, encoder


def make_submission(y_prob, ids, encoder, fname):
    with open(fname, 'w') as f:
        f.write('id,')
        f.write(','.join([str(i) for i in encoder.classes_]))
        f.write('\n')
        for i, probs in zip(ids, y_prob):
            probas = ','.join([i] + [str(p) for p in probs.tolist()])
            f.write(probas)
            f.write('\n')
    print("Wrote submission to file {}.".format(fname))

In [22]:
print("Loading data...")
X, labels = load_data('otto/train.csv', train=True)
X, scaler = preprocess_data(X)
y, encoder = preprocess_labels(labels)

X_test, ids = load_data('otto/test.csv', train=False)
X_test, _ = preprocess_data(X_test, scaler)

nb_classes = y.shape[1]
print(nb_classes, 'classes')

dims = X.shape[1]
print(dims, 'dims')



Loading data...
9 classes
93 dims


In [23]:
print("Building model...")
model = Sequential()
model.add(Dense(dims, 512, init='glorot_uniform'))
model.add(PReLU((512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(512, 512, init='glorot_uniform'))
model.add(PReLU((512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(512, 512, init='glorot_uniform'))
model.add(PReLU((512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(512, nb_classes, init='glorot_uniform'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam")


Building model...


In [33]:
print (X)
print (y)

[[-0.25351954 -0.21006845 -0.30717063 ..., -0.12956563 -0.38694546
  -0.10497295]
 [-0.25351954 -0.21006845 -0.30717063 ..., -0.12956563 -0.38694546
  -0.10497295]
 [-0.25351954 -0.21006845  1.39655519 ..., -0.12956563  0.63101345
  -0.10497295]
 ..., 
 [-0.25351954 -0.21006845 -0.30717063 ..., -0.12956563 -0.38694546
  -0.10497295]
 [-0.25351954 -0.21006845 -0.30717063 ..., -0.12956563 -0.38694546
  -0.10497295]
 [-0.25351954 -0.21006845 -0.30717063 ..., -0.12956563 -0.38694546
  -0.10497295]]
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [ ]:

print("Training model...")

model.fit(X, y, nb_epoch=20, batch_size=128, validation_split=0.15,callbacks=[remote],show_accuracy=True)


Training model...
Train on 52596 samples, validate on 9282 samples
Epoch 0
52596/52596 [==============================] - 33s - loss: 0.4317 - acc: 0.8267 - val_loss: 0.4980 - val_acc: 0.8073
Epoch 1
 9088/52596 [====>.........................] - ETA: 29s - loss: 0.4269 - acc: 0.8283

In [43]:
model.save_weights("otto1.hdf5",overwrite=True)
f = h5py.File('otto1.hdf5','r')
print("Generating submission...")

proba = model.predict_proba(X_test)
print(proba)
make_submission(proba, ids, encoder, fname='keras-otto.csv')

Generating submission...
144368/144368 [==============================] - 11s    
[[  1.46647332e-04   3.89051336e-01   3.08881463e-01 ...,   3.90837949e-03
    1.01941808e-04   9.17399650e-05]
 [  2.73822839e-04   1.26874818e-04   5.07051475e-05 ...,   5.44620673e-04
    4.09600495e-01   5.26564298e-04]
 [  7.10318091e-10   1.02060842e-11   4.75442326e-12 ...,   3.04166051e-10
    2.87826749e-06   6.20159396e-08]
 ..., 
 [  9.44453122e-05   3.56525610e-01   5.47618001e-01 ...,   6.66131798e-03
    1.67525110e-03   7.91098851e-04]
 [  1.37764686e-04   5.46825135e-01   5.57123240e-02 ...,   2.94399418e-04
    2.70151923e-05   1.92963564e-04]
 [  2.53428451e-04   4.69088364e-01   4.38677559e-01 ...,   2.55371342e-02
    2.64639969e-04   1.54532527e-04]]
Wrote submission to file keras-otto.csv.
